In [1]:
import numpy as np
import pandas as pd
import pickle

def save_obj(obj, loc ):
    with open(loc, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(loc):
    with open(loc, 'rb') as f:
        return pickle.load(f)

def reverse_dict(my_map):
    inv_map = {v: k for k, v in my_map.items()}
    return inv_map

reg_2_num = load_obj('/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')

comm_2_num = load_obj('/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')

prodcomm_2_num = load_obj('/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/prodcomm_2_num.pickle')
nsavcomm_2_num = load_obj('/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/nsavcomm_2_num.pickle')
demdcomm_2_num = load_obj('/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/demdcomm_2_num.pickle')

In [2]:
root = '/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/matrices/{}.pickle'

In [3]:
F_goods = ['b_t','tex','wap','lea','lum','ppp','p_c','crp','nmm','fmp','mvh','otn','ele',\
           'ome','omf','ely','gdt','wtr','cns','trd','otp','wtp','atp','cmn','ofi','isr','obs','ros','osg']

C_goods = ['pdr','wht','gro','v_f','osd','c_b','pfb','ocr','ctl','oap','rmk','wol','frs','fsh','coa','oil',\
           'gas','omn','cmt','omt','vol','mil','pcr','sgr','i_s','nfm','ofd']

In [4]:
len(F_goods)+len(C_goods)

56

In [5]:
data = {}
n = 134
g = len(C_goods)
k = len(F_goods)

data['n'] = n
data['g'] = g
data['k'] = k

In [6]:
# Lets get the easy one out of the way
data['E'] = load_obj(root.format('E')).reshape((n, 1))

In [7]:
# Now production:
Y = load_obj(root.format('Y'))
data['Y_g'] = np.zeros((n, g))
data['Y_k'] = np.zeros((n, k))


for i, key in enumerate(C_goods):
    data['Y_g'][:, i] = Y[comm_2_num[key], :]


for i, key in enumerate(F_goods):
    data['Y_k'][:, i] = Y[comm_2_num[key], :]

In [8]:
# Now demand:
D = load_obj(root.format('D'))
data['D_g_old'] = np.zeros((n, g))
data['D_k_old'] = np.zeros((n, k))


for i, key in enumerate(C_goods):
    data['D_g_old'][:, i] = D[comm_2_num[key], :]


for i, key in enumerate(F_goods):
    data['D_k_old'][:, i] = D[comm_2_num[key], :]

In [9]:
# Now trade
X = load_obj(root.format('X'))

data['X_g'] = np.zeros((n, n, g))
data['X_k'] = np.zeros((n, n, k))


for i, key in enumerate(C_goods):
    data['X_g'][:, :, i] = X[comm_2_num[key], :, :].T


for i, key in enumerate(F_goods):
    data['X_k'][:, :, i] = X[comm_2_num[key], :, :].T

In [10]:
# Adjust local consumption
for i in range(n):
    temp = np.zeros(g)
    for j in range(n):
        if i != j:
            temp += data['X_g'][j, i, :]
        
    data['X_g'][i, i, :] = data['Y_g'][i,:] - temp

# Re-adjust with demand figures
idx_g = np.where(data['X_g'] <= 0)
data['X_g'][idx_g] = data['D_g_old'][idx_g[1:]]

# Now go back 
for i in range(idx_g[0].shape[0]):
    print(idx_g[1][i], idx_g[2][i])
    data['Y_g'][idx_g[1][i], idx_g[2][i]] += data['X_g'][:, idx_g[1][i], idx_g[2][i]].sum()

7 15
17 15
115 15
122 25
132 15


In [11]:
for i in range(n):
    temp = np.zeros(k)
    for j in range(n):
        if i != j:
            temp += data['X_k'][j, i, :]
        
    data['X_k'][i, i, :] = data['Y_k'][i,:] - temp

idx_k = np.where(data['X_k'] <= 0)
data['X_k'][idx_k] = data['D_k_old'][idx_k[1:]]

 
for i in range(idx_k[0].shape[0]):
    print(idx_k[1][i], idx_k[2][i])
    data['Y_g'][idx_k[1][i], idx_k[2][i]] += data['X_k'][:, idx_k[1][i], idx_k[2][i]].sum()

86 4
113 1
114 11


In [12]:
data['D_g'] = data['X_g'].sum(axis=1)
data['D_k'] = data['X_k'].sum(axis=1)

In [13]:
# Trade fraction
data['lambda_g'] = data['X_g']/data['D_g'].reshape((n, 1, g))
data['lambda_k'] = data['X_k']/data['D_k'].reshape((n, 1, k))

In [14]:
# Now interlinkage
d = load_obj(root.format('d_l'))

data['d'] = np.zeros((n, k, g))

for i, comm in enumerate(C_goods):
    for j, final in enumerate(F_goods):
        data['d'][:, j, i] = d[comm_2_num[comm], comm_2_num[final], :]

In [15]:
# Share of labor
e_labor = load_obj(root.format('e_labor'))

data['e_L'] = e_labor.reshape((n, 1))


# Share of commodities
e_comm = load_obj(root.format('e_comm'))
data['e'] = np.zeros((n, g))

for i, key in enumerate(C_goods):
    data['e'][:, i] = e_comm[comm_2_num[key], :]


In [16]:
data['e_L'] =( 1 - data['e'].sum(axis=1)).reshape((n, 1))

In [20]:
data['e'].sum(axis=1)

array([0.27835701, 0.3294482 , 0.41913555, 0.56087113, 0.18008043,
       0.1475382 , 0.21798218, 0.64649758, 0.22391398, 0.23915743,
       0.64012794, 0.50648693, 1.00171498, 0.47306545, 0.39112444,
       0.03917416, 0.3326907 , 0.63301321, 0.7763111 , 0.62158066,
       0.42483334, 0.49062759, 0.48754147, 0.39213039, 0.48794164,
       0.22421672, 0.10877491, 0.25494029, 0.16055771, 0.37507264,
       0.67248152, 0.2875467 , 0.55101063, 0.32102221, 0.66282989,
       0.67018581, 0.58789847, 0.31057899, 0.48380186, 0.61472596,
       0.38345971, 0.48012966, 0.51937406, 0.61053925, 0.2982315 ,
       0.36007275, 0.50201007, 0.24998124, 0.12861005, 0.22014831,
       0.19989755, 0.2393051 , 0.16523011, 0.27001176, 0.18403446,
       0.13951954, 0.11332903, 0.18601292, 0.24586698, 0.14783745,
       0.15093358, 0.37063091, 0.26544557, 0.16839146, 0.08925727,
       0.16877676, 0.29998648, 0.15165387, 0.31322973, 0.19830619,
       0.14877962, 0.12462032, 0.10086627, 0.14128976, 0.31703

In [42]:
# alpha
alpha = load_obj(root.format('alpha'))
data['alpha'] = np.zeros((n, k))

for i, key in enumerate(F_goods):
    data['alpha'][:, i] = alpha[comm_2_num[key], :]

In [43]:
idx_k

(array([ 86, 113, 114]), array([ 86, 113, 114]), array([ 4,  1, 11]))

In [44]:
idx_g

(array([  7,  17, 115, 122, 132]),
 array([  7,  17, 115, 122, 132]),
 array([15, 15, 15, 25, 15]))

In [45]:
# phi
phi_labor = load_obj(root.format('phi_labor'))
phi_comm = load_obj(root.format('phi_comm'))
phi_natlres = load_obj(root.format('phi_natlres')) # See if you need to integrate land


data['phi_L_g'] = np.zeros((n, g))
data['phi_L_k'] = np.zeros((n, k))

data['phi_R'] = np.zeros((n, g))

data['phi'] = np.zeros((n, k, g))

for i, key in enumerate(C_goods):
    data['phi_L_g'][:, i] = phi_labor[comm_2_num[key], :]
    data['phi_R'][:, i] = phi_natlres[comm_2_num[key], :]
    
for i, key in enumerate(F_goods):    
    data['phi_L_k'][:, i] = phi_labor[comm_2_num[key], :]
    
for i, comm in enumerate(C_goods):
    for j, final in enumerate(F_goods):
        data['phi'][:, j, i] = d[comm_2_num[comm], comm_2_num[final], :]

In [46]:
# Update based on notes:
data['phi'] = data['d']*data['D_g'].reshape((n, 1, g))/data['Y_k'].reshape((n, k, 1))

data['phi_L_k'] = 1 - data['phi'].sum(axis=2)

print(np.where(data['phi'] >1), np.where(data['phi_L_k'] <=0))


(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)) (array([], dtype=int64), array([], dtype=int64))
3.0917156944473025e-08 308.94022842087634 15.917555809000001


In [47]:
data['theta_k'] = np.ones((k, 1))*5.
data['theta_g'] = np.ones((g, 1))*5.

data['eta'] = np.ones((k, 1))*0.4
data['rho_g'] = np.ones((g, 1))*0.4

data['sigma'] = 0.4

In [48]:
### Change non zero elements to zero ###

In [49]:
for i in data.keys():
    print(i)
    if i not in ['n', 'g','k','sigma']:
        data[i][data[i]<0] = 0

n
g
k
E
Y_g
Y_k
D_g_old
D_k_old
X_g
X_k
D_g
D_k
lambda_g
lambda_k
d
e_L
e
alpha
phi_L_g
phi_L_k
phi_R
phi
theta_k
theta_g
eta
rho_g
sigma


In [50]:
loc = '/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/final_data/data_20200212.pickle'
save_obj(data, loc)